# 🕷️ Lección 5: Introducción a Scrapy Framework

## 🎯 Objetivos

- Entender la arquitectura de Scrapy
- Crear y configurar proyectos Scrapy
- Desarrollar spiders básicos y avanzados
- Usar Items y Pipelines
- Configurar settings y middlewares
- Manejar requests y responses eficientemente

## 🏗️ ¿Qué es Scrapy?

**Scrapy** es el framework más potente y completo para web scraping en Python. A diferencia de Beautiful Soup que es una biblioteca, Scrapy es un framework completo con:

### 🌟 Ventajas de Scrapy:
- **Asíncrono**: Maneja múltiples requests simultáneamente
- **Robusto**: Manejo automático de errores y reintentos
- **Escalable**: Perfecto para proyectos grandes
- **Extensible**: Middlewares y pipelines personalizables
- **Integrado**: Exportación automática a JSON, CSV, XML

### 🏛️ Arquitectura de Scrapy:
```
Engine ← → Scheduler ← → Downloader
   ↕              ↕         ↕
Spider ← → Items → Item Pipeline
```

In [ ]:
# Instalación y configuración inicial
import scrapy
from scrapy import Request, Spider
from scrapy.http import Response
import json
import pandas as pd

print("🕷️ SCRAPY FRAMEWORK INTRODUCTION")
print("=" * 40)
print(f"Scrapy version: {scrapy.__version__}")
print("✅ Scrapy importado correctamente")

## 🚀 Creando tu Primer Spider

In [ ]:
# Spider básico para Quotes to Scrape
class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']

    def parse(self, response):
        """Método principal de parsing"""
        print(f"🌐 Procesando: {response.url}")
        
        # Extraer todas las citas
        quotes = response.css('.quote')
        print(f"📖 Encontradas {len(quotes)} citas")
        
        for quote in quotes:
            # Extraer datos con CSS selectors
            yield {
                'text': quote.css('.text::text').get(),
                'author': quote.css('.author::text').get(),
                'tags': quote.css('.tag::text').getall(),
            }
        
        # Seguir a la siguiente página
        next_page = response.css('.next a::attr(href)').get()
        if next_page:
            print(f"➡️ Siguiente página: {next_page}")
            yield response.follow(next_page, self.parse)

print("🕷️ SPIDER BÁSICO CREADO")
print("Componentes del Spider:")
print("  • name: Identificador único")
print("  • start_urls: URLs iniciales")
print("  • parse(): Método de procesamiento")
print("  • yield: Retorna datos o requests")

## 🏷️ Items y Pipelines

In [ ]:
# Definir Items (estructura de datos)
import scrapy
from scrapy import Item, Field

class QuoteItem(scrapy.Item):
    """Estructura para almacenar citas"""
    text = scrapy.Field()
    author = scrapy.Field()
    tags = scrapy.Field()
    url = scrapy.Field()
    scraped_at = scrapy.Field()

class ProductItem(scrapy.Item):
    """Estructura para productos e-commerce"""
    name = scrapy.Field()
    price = scrapy.Field()
    description = scrapy.Field()
    images = scrapy.Field()
    availability = scrapy.Field()
    rating = scrapy.Field()
    reviews_count = scrapy.Field()

print("🏷️ ITEMS DEFINIDOS")
print("Items actúan como estructuras de datos validadas")

# Pipeline básico
class ValidationPipeline:
    """Pipeline para validar datos"""
    
    def process_item(self, item, spider):
        if not item.get('text'):
            raise scrapy.exceptions.DropItem("Texto faltante")
        
        if not item.get('author'):
            raise scrapy.exceptions.DropItem("Autor faltante")
        
        return item

class ProcessingPipeline:
    """Pipeline para procesar datos"""
    
    def process_item(self, item, spider):
        # Limpiar texto
        if item.get('text'):
            item['text'] = item['text'].strip('"')
        
        # Convertir tags a minúsculas
        if item.get('tags'):
            item['tags'] = [tag.lower() for tag in item['tags']]
        
        # Agregar timestamp
        from datetime import datetime
        item['scraped_at'] = datetime.now().isoformat()
        
        return item

print("\n🔧 PIPELINES CREADOS")
print("  • ValidationPipeline: Valida datos requeridos")
print("  • ProcessingPipeline: Limpia y procesa datos")

## 🕷️ Spider Avanzado con Items

In [ ]:
class AdvancedQuotesSpider(scrapy.Spider):
    name = 'advanced_quotes'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']
    
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ValidationPipeline': 300,
            '__main__.ProcessingPipeline': 400,
        },
        'DOWNLOAD_DELAY': 1,  # Pausa entre requests
        'RANDOMIZE_DOWNLOAD_DELAY': 0.5,
        'USER_AGENT': 'advanced_quotes_spider'
    }

    def parse(self, response):
        """Parse principal con Items"""
        self.logger.info(f'Parseando página: {response.url}')
        
        # Extraer citas usando Items
        quotes = response.css('.quote')
        
        for quote in quotes:
            item = QuoteItem()
            item['text'] = quote.css('.text::text').get()
            item['author'] = quote.css('.author::text').get()
            item['tags'] = quote.css('.tag::text').getall()
            item['url'] = response.url
            
            yield item
            
            # También obtener información del autor
            author_url = quote.css('.author + a::attr(href)').get()
            if author_url:
                yield response.follow(
                    author_url, 
                    self.parse_author,
                    meta={'author_name': item['author']}
                )
        
        # Paginación
        next_page = response.css('.next a::attr(href)').get()
        if next_page:
            yield response.follow(next_page, self.parse)
    
    def parse_author(self, response):
        """Parse información de autores"""
        author_name = response.meta['author_name']
        
        yield {
            'author_name': author_name,
            'born_date': response.css('.author-born-date::text').get(),
            'born_location': response.css('.author-born-location::text').get(),
            'description': response.css('.author-description::text').get(),
        }

print("🕷️ SPIDER AVANZADO CREADO")
print("Características avanzadas:")
print("  • Custom settings por spider")
print("  • Items estructurados")
print("  • Múltiples callbacks (parse_author)")
print("  • Logging integrado")
print("  • Meta data entre requests")

## ⚙️ Configuración y Settings

In [ ]:
# Simulación de settings.py
SCRAPY_SETTINGS = {
    # Identificación del bot
    'BOT_NAME': 'mi_scraper',
    'USER_AGENT': 'mi_scraper (+http://www.yourdomain.com)',
    
    # Configuración de cortesía
    'ROBOTSTXT_OBEY': True,
    'DOWNLOAD_DELAY': 3,
    'RANDOMIZE_DOWNLOAD_DELAY': 0.5,
    'CONCURRENT_REQUESTS': 16,
    'CONCURRENT_REQUESTS_PER_DOMAIN': 8,
    
    # Pipelines
    'ITEM_PIPELINES': {
        'myproject.pipelines.ValidationPipeline': 300,
        'myproject.pipelines.DuplicatesPipeline': 400,
        'myproject.pipelines.DatabasePipeline': 500,
    },
    
    # Middlewares
    'DOWNLOADER_MIDDLEWARES': {
        'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
        'scrapy_user_agents.middlewares.RandomUserAgentMiddleware': 400,
        'scrapy.downloadermiddlewares.retry.RetryMiddleware': 500,
    },
    
    # Configuración de reintentos
    'RETRY_TIMES': 3,
    'RETRY_HTTP_CODES': [500, 502, 503, 504, 408, 429],
    
    # Exportación
    'FEEDS': {
        'output/quotes.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
        },
        'output/quotes.csv': {
            'format': 'csv',
            'encoding': 'utf8',
        },
    },
    
    # Logging
    'LOG_LEVEL': 'INFO',
    'LOG_FILE': 'scrapy.log',
    
    # Cache
    'HTTPCACHE_ENABLED': True,
    'HTTPCACHE_EXPIRATION_SECS': 3600,
    'HTTPCACHE_DIR': 'httpcache',
}

print("⚙️ CONFIGURACIÓN DE SCRAPY")
print("=" * 35)
print("Configuraciones importantes:")
print(f"  🤖 Bot Name: {SCRAPY_SETTINGS['BOT_NAME']}")
print(f"  ⏱️ Download Delay: {SCRAPY_SETTINGS['DOWNLOAD_DELAY']}s")
print(f"  🔄 Concurrent Requests: {SCRAPY_SETTINGS['CONCURRENT_REQUESTS']}")
print(f"  📊 Pipelines: {len(SCRAPY_SETTINGS['ITEM_PIPELINES'])}")
print(f"  📁 Export Formats: {len(SCRAPY_SETTINGS['FEEDS'])}")
print(f"  💾 Cache Enabled: {SCRAPY_SETTINGS['HTTPCACHE_ENABLED']}")

## 🛠️ Middlewares Personalizados

In [ ]:
# Middleware para rotar User Agents
class RotateUserAgentMiddleware:
    def __init__(self):
        self.user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36',
        ]
        self.current_ua = 0
    
    def process_request(self, request, spider):
        ua = self.user_agents[self.current_ua]
        request.headers['User-Agent'] = ua
        self.current_ua = (self.current_ua + 1) % len(self.user_agents)
        return None

# Middleware para manejar proxies
class ProxyMiddleware:
    def __init__(self):
        self.proxies = [
            'http://proxy1:8000',
            'http://proxy2:8000',
            'http://proxy3:8000',
        ]
        self.current_proxy = 0
    
    def process_request(self, request, spider):
        proxy = self.proxies[self.current_proxy]
        request.meta['proxy'] = proxy
        self.current_proxy = (self.current_proxy + 1) % len(self.proxies)
        return None

# Middleware para manejar errores
class ErrorHandlingMiddleware:
    def process_response(self, request, response, spider):
        if response.status in [403, 404, 429]:
            spider.logger.warning(f'Error {response.status} en {request.url}')
            # Crear nuevo request con delay
            new_request = request.copy()
            new_request.meta['delay'] = 10
            return new_request
        return response
    
    def process_exception(self, request, exception, spider):
        spider.logger.error(f'Excepción en {request.url}: {exception}')
        # Reintentar con delay
        new_request = request.copy()
        new_request.meta['retry_times'] = request.meta.get('retry_times', 0) + 1
        if new_request.meta['retry_times'] < 3:
            return new_request
        return None

print("🛠️ MIDDLEWARES PERSONALIZADOS")
print("=" * 35)
print("Middlewares disponibles:")
print("  🔄 RotateUserAgentMiddleware: Rota User Agents")
print("  🌐 ProxyMiddleware: Maneja proxies")
print("  ❌ ErrorHandlingMiddleware: Maneja errores")

## 🏪 Spider para E-commerce

In [ ]:
class EcommerceSpider(scrapy.Spider):
    name = 'books'
    allowed_domains = ['books.toscrape.com']
    start_urls = ['http://books.toscrape.com/']
    
    custom_settings = {
        'DOWNLOAD_DELAY': 1,
        'CONCURRENT_REQUESTS_PER_DOMAIN': 2,
        'FEEDS': {
            'books.json': {
                'format': 'json',
                'encoding': 'utf8',
                'indent': 2
            }
        }
    }

    def parse(self, response):
        """Parse página principal y categorías"""
        # Obtener todos los libros de la página
        books = response.css('article.product_pod')
        
        for book in books:
            book_url = book.css('h3 a::attr(href)').get()
            if book_url:
                yield response.follow(
                    book_url, 
                    self.parse_book,
                    meta={'category': self.get_category_from_breadcrumb(response)}
                )
        
        # Seguir paginación
        next_page = response.css('.next a::attr(href)').get()
        if next_page:
            yield response.follow(next_page, self.parse)
    
    def parse_book(self, response):
        """Parse página individual del libro"""
        # Crear item del producto
        item = ProductItem()
        
        # Información básica
        item['name'] = response.css('h1::text').get()
        item['price'] = response.css('.price_color::text').get()
        item['availability'] = response.css('.availability::text').re_first(r'In stock \((\d+) available\)')
        item['description'] = response.css('#product_description + p::text').get()
        
        # Rating (convertir estrellas a número)
        rating_class = response.css('.star-rating::attr(class)').get()
        if rating_class:
            rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
            rating_text = rating_class.replace('star-rating ', '')
            item['rating'] = rating_map.get(rating_text, 0)
        
        # Información adicional de la tabla
        table_rows = response.css('table tr')
        for row in table_rows:
            field = row.css('td:first-child::text').get()
            value = row.css('td:last-child::text').get()
            
            if field == 'Number of reviews':
                item['reviews_count'] = int(value) if value.isdigit() else 0
        
        # Imágenes
        image_url = response.css('#product_gallery img::attr(src)').get()
        if image_url:
            item['images'] = [response.urljoin(image_url)]
        
        # Categoría desde meta
        category = response.meta.get('category', 'Unknown')
        
        # Agregar datos calculados
        price_text = item.get('price', '£0.00')
        price_numeric = float(price_text.replace('£', '').replace(',', '')) if price_text else 0
        
        yield {
            **item,
            'category': category,
            'price_numeric': price_numeric,
            'url': response.url,
            'scraped_at': self.get_timestamp()
        }
    
    def get_category_from_breadcrumb(self, response):
        """Extraer categoría del breadcrumb"""
        breadcrumb = response.css('.breadcrumb li:last-child::text').get()
        return breadcrumb.strip() if breadcrumb else 'All'
    
    def get_timestamp(self):
        """Obtener timestamp actual"""
        from datetime import datetime
        return datetime.now().isoformat()

print("🏪 SPIDER E-COMMERCE CREADO")
print("Características del spider:")
print("  📚 Scraping de libros completo")
print("  🔗 Seguimiento de enlaces de productos")
print("  📊 Extracción de ratings y precios")
print("  📱 Manejo de imágenes y metadatos")
print("  📄 Paginación automática")

## 🎯 Ejercicios Prácticos

In [ ]:
print("🎯 EJERCICIOS SCRAPY")
print("=" * 25)

# Ejercicio 1: Spider básico
class NewsSpider(scrapy.Spider):
    """Ejercicio: Crear spider para noticias"""
    name = 'news'
    start_urls = ['https://example-news-site.com']
    
    def parse(self, response):
        # Extraer títulos de noticias
        articles = response.css('.article')
        
        for article in articles:
            yield {
                'title': article.css('.title::text').get(),
                'summary': article.css('.summary::text').get(),
                'date': article.css('.date::text').get(),
                'author': article.css('.author::text').get(),
                'url': response.url
            }

# Ejercicio 2: Pipeline de limpieza
class CleaningPipeline:
    """Pipeline para limpiar y validar noticias"""
    
    def process_item(self, item, spider):
        # Limpiar título
        if item.get('title'):
            item['title'] = item['title'].strip().title()
        
        # Validar fecha
        if not item.get('date'):
            from datetime import datetime
            item['date'] = datetime.now().strftime('%Y-%m-%d')
        
        # Calcular longitud del resumen
        if item.get('summary'):
            item['summary_length'] = len(item['summary'])
        
        return item

# Ejercicio 3: Spider con formularios
class SearchSpider(scrapy.Spider):
    """Spider que maneja formularios de búsqueda"""
    name = 'search'
    start_urls = ['https://example-search.com']
    
    def parse(self, response):
        # Llenar formulario de búsqueda
        return scrapy.FormRequest.from_response(
            response,
            formdata={'query': 'python scrapy'},
            callback=self.parse_results
        )
    
    def parse_results(self, response):
        # Procesar resultados de búsqueda
        results = response.css('.search-result')
        
        for result in results:
            yield {
                'title': result.css('.title::text').get(),
                'link': result.css('a::attr(href)').get(),
                'snippet': result.css('.snippet::text').get()
            }

print("📝 EJERCICIOS CREADOS:")
print("  1. 📰 NewsSpider - Spider para noticias")
print("  2. 🧹 CleaningPipeline - Limpieza de datos")
print("  3. 🔍 SearchSpider - Formularios y búsquedas")

# Simulación de ejecución
print("\n🚀 SIMULACIÓN DE EJECUCIÓN:")
print("Comandos típicos de Scrapy:")
print("  scrapy startproject mi_proyecto")
print("  scrapy genspider quotes quotes.toscrape.com")
print("  scrapy crawl quotes -o quotes.json")
print("  scrapy shell 'http://quotes.toscrape.com'")
print("  scrapy list  # Ver spiders disponibles")
print("  scrapy check quotes  # Verificar spider")

## 📊 Ejemplo Completo: Spider + Pipeline + Export

In [ ]:
# Ejemplo completo integrado
class ComprehensiveSpider(scrapy.Spider):
    """Spider completo con todas las características"""
    name = 'comprehensive'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']
    
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ValidationPipeline': 300,
            '__main__.ProcessingPipeline': 400,
        },
        'FEEDS': {
            'comprehensive_output.json': {
                'format': 'json',
                'encoding': 'utf8',
                'indent': 2
            },
            'comprehensive_output.csv': {
                'format': 'csv',
                'encoding': 'utf8'
            }
        },
        'DOWNLOAD_DELAY': 1,
        'RANDOMIZE_DOWNLOAD_DELAY': 0.5,
        'USER_AGENT': 'comprehensive_spider 1.0'
    }
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stats = {
            'quotes_scraped': 0,
            'authors_scraped': 0,
            'pages_processed': 0,
            'errors': 0
        }
    
    def parse(self, response):
        """Parse principal con estadísticas"""
        self.stats['pages_processed'] += 1
        self.logger.info(f"Procesando página {self.stats['pages_processed']}: {response.url}")
        
        quotes = response.css('.quote')
        
        for quote in quotes:
            try:
                item = QuoteItem()
                item['text'] = quote.css('.text::text').get()
                item['author'] = quote.css('.author::text').get()
                item['tags'] = quote.css('.tag::text').getall()
                item['url'] = response.url
                
                self.stats['quotes_scraped'] += 1
                yield item
                
                # Obtener info del autor
                author_url = quote.css('.author + a::attr(href)').get()
                if author_url:
                    yield response.follow(
                        author_url,
                        self.parse_author,
                        meta={'author_name': item['author']}
                    )
                    
            except Exception as e:
                self.stats['errors'] += 1
                self.logger.error(f"Error procesando cita: {e}")
        
        # Paginación
        next_page = response.css('.next a::attr(href)').get()
        if next_page and self.stats['pages_processed'] < 5:  # Limitar para demo
            yield response.follow(next_page, self.parse)
        else:
            self.logger.info("Scraping completado - Estadísticas finales:")
            for key, value in self.stats.items():
                self.logger.info(f"  {key}: {value}")
    
    def parse_author(self, response):
        """Parse información detallada de autores"""
        try:
            self.stats['authors_scraped'] += 1
            
            yield {
                'type': 'author',
                'name': response.meta['author_name'],
                'born_date': response.css('.author-born-date::text').get(),
                'born_location': response.css('.author-born-location::text').get(),
                'description': response.css('.author-description::text').get(),
                'url': response.url
            }
            
        except Exception as e:
            self.stats['errors'] += 1
            self.logger.error(f"Error procesando autor: {e}")

print("📊 SPIDER COMPREHENSIVO CREADO")
print("Características completas:")
print("  📈 Estadísticas en tiempo real")
print("  📝 Logging detallado")
print("  🔄 Manejo de errores robusto")
print("  📤 Export múltiple (JSON + CSV)")
print("  ⚙️ Settings personalizados")
print("  🕷️ Multi-callback processing")

# Simulación de resultados
print("\n📋 EJEMPLO DE RESULTADOS:")
sample_results = {
    'quotes_scraped': 50,
    'authors_scraped': 25,
    'pages_processed': 5,
    'errors': 2
}

for key, value in sample_results.items():
    print(f"  {key}: {value}")

## 📖 Resumen de la Lección

### 🎯 Lo que Hemos Dominado

1. **Fundamentos de Scrapy**:
   - Arquitectura del framework
   - Componentes principales (Engine, Scheduler, Downloader)
   - Diferencias con Beautiful Soup

2. **Spiders Avanzados**:
   - Creación de spiders básicos y avanzados
   - Manejo de múltiples callbacks
   - Paginación automática
   - Extracción de datos complejos

3. **Items y Pipelines**:
   - Definición de estructuras de datos
   - Validación automática
   - Procesamiento de datos
   - Pipelines de limpieza y transformación

4. **Configuración Avanzada**:
   - Settings personalizados
   - Middlewares custom
   - Manejo de errores y reintentos
   - Configuración de cortesía

5. **Casos de Uso Reales**:
   - Spider para e-commerce
   - Manejo de formularios
   - Extracción de metadatos
   - Export automático

### 🚀 Próxima Lección: Spiders Avanzados

En la siguiente lección profundizaremos en:
- Spiders especializados (CrawlSpider, XMLFeedSpider)
- Manejo de JavaScript con Splash
- Scraping distribuido con Scrapyd
- Debugging y optimización
- Casos de uso complejos

### 💡 Comandos Esenciales

```bash
# Crear proyecto
scrapy startproject myproject

# Generar spider
scrapy genspider quotes quotes.toscrape.com

# Ejecutar spider
scrapy crawl quotes -o output.json

# Shell interactivo
scrapy shell "http://quotes.toscrape.com"

# Verificar spider
scrapy check quotes
```

---

¡Excelente progreso! 🎉 Ahora dominas Scrapy y puedes crear scrapers industriales robustos y escalables.